# using the 5 types of flower dataset from kaggle

importing the necessary packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
# Set the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Define the data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image pixels
])

In [4]:
# Load the dataset
dataset = ImageFolder('/kaggle/input/5-flower-types-classification-dataset', transform=transform)

In [12]:
# Split the dataset into training and testing sets
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [13]:
# Create data loaders for batching the data
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
class FlowerClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FlowerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 56 * 56, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [18]:
# Define the hyperparameters
num_classes = len(dataset.classes)
learning_rate = 0.01
num_epochs = 10

In [19]:
# Initialize the model, loss function, and optimizer
model = FlowerClassifier(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
         # Print the loss for every 100 batches
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/110], Loss: 0.0000
Epoch [2/10], Step [100/110], Loss: 0.0000
Epoch [3/10], Step [100/110], Loss: 0.0000
Epoch [4/10], Step [100/110], Loss: 0.0000
Epoch [5/10], Step [100/110], Loss: 0.0000
Epoch [6/10], Step [100/110], Loss: 0.0000
Epoch [7/10], Step [100/110], Loss: 0.0000
Epoch [8/10], Step [100/110], Loss: 0.0000
Epoch [9/10], Step [100/110], Loss: 0.0000
Epoch [10/10], Step [100/110], Loss: 0.0000


In [21]:
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {(correct / total) * 100:.2f}%')

Test Accuracy: 100.00%
